# Sesiunea 17 – Pandas Avansat: Time Series, Pivot Tables, Missing Data și Pipeline-uri de Procesare
_Notebook de exerciții (fără soluții)._

## Exercițiul 1 — DataFrame cu date zilnice
- Creează un `DataFrame` cu **date zilnice** folosind `pd.date_range` pe un interval la alegere (ex: `2023-01-01` → `2023-03-31`, `freq='D'`).
- Include coloane relevante pentru temă: `Data` (sau setează ca index), `Oraș`, `Produs`, `Vânzări`.
- **Cerințe:**
  1. Setează indexul pe data calendaristică (`DatetimeIndex`).
  2. Verifică tipurile de date și asigură-te că data este de tip `datetime64[ns]`.
  3. Afișează 5 rânduri de început și 5 de final pentru inspecție.

In [ ]:
import pandas as pd
import random

dates = pd.date_range("2023-01-01", "2023-03-31", freq='D')

cities = [
    "New York", "Philadelphia", "Boston", "Washington D.C.",
    "Baltimore", "Pittsburgh", "Cleveland", "Buffalo",
    "Hartford", "Providence"]

products = [
    "Laptop", "Desktop PC", "Smartphone", "Tablet",
    "Smartwatch", "Headphones", "Bluetooth Speaker",
    "Keyboard", "Mouse", "Monitor", "Webcam", "Printer"]

data = {
    "City": [random.choice(cities) for _ in range(len(dates))],
    "Product": [random.choice(products) for _ in range(len(dates))],
    "Sales": [random.randint(1, 25) for _ in range(len(dates))]
}

df = pd.DataFrame(data, index=dates)

print(df.index.dtype)

df

datetime64[ns]


,City,Product,Sales
2023-01-01,Buffalo,Webcam,11
2023-01-02,Pittsburgh,Monitor,8
2023-01-03,Baltimore,Mouse,22
2023-01-04,Baltimore,Monitor,5
2023-01-05,Providence,Desktop PC,25
...,...,...,...
2023-03-27,Hartford,Webcam,19
2023-03-28,Philadelphia,Smartphone,9
2023-03-29,Hartford,Webcam,9
2023-03-30,Cleveland,Headphones,7


## Exercițiul 2 — Filtrare o săptămână
- Folosind `DatetimeIndex`, **filtrează** exact o **săptămână** (ex: `2023-02-06` → `2023-02-12`).
- Arată două metode: **slice** pe index (ex: `df.loc['YYYY-MM-DD':'YYYY-MM-DD']`) și filtrare cu condiții booleene.
- **Cerințe:**
  1. Afișează `shape` și statistici descriptive pentru intervalul selectat.
  2. Verifică ordonarea cronologică și lipsa datelor duplicate pe dată.

In [153]:
week_slice = df.loc["2023-02-06":"2023-02-12"]
print("shape:", week_slice.shape)
print(week_slice.describe(include='all'))
print()
print("is_monotonic_increasing", df.index.is_monotonic_increasing)
print("is_unique", df.index.is_unique)
week_slice

shape: (7, 3)
             City  Product      Sales
count           7        7   7.000000
unique          4        5        NaN
top     Cleveland  Printer        NaN
freq            2        2        NaN
mean          NaN      NaN  10.571429
std           NaN      NaN   6.704654
min           NaN      NaN   1.000000
25%           NaN      NaN   5.000000
50%           NaN      NaN  15.000000
75%           NaN      NaN  16.000000
max           NaN      NaN  16.000000

is_monotonic_increasing True
is_unique True


,City,Product,Sales
2023-02-06,Cleveland,Printer,7
2023-02-07,Pittsburgh,Smartphone,15
2023-02-08,Boston,Keyboard,16
2023-02-09,Pittsburgh,Printer,3
2023-02-10,Baltimore,Smartphone,16
2023-02-11,Boston,Smartwatch,1
2023-02-12,Cleveland,Laptop,16


In [154]:
mask = (df.index >= "2023-02-06") & (df.index <= "2023-02-12")
mask

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [155]:
week_filter = df[mask]
week_filter

,City,Product,Sales
2023-02-06,Cleveland,Printer,7
2023-02-07,Pittsburgh,Smartphone,15
2023-02-08,Boston,Keyboard,16
2023-02-09,Pittsburgh,Printer,3
2023-02-10,Baltimore,Smartphone,16
2023-02-11,Boston,Smartwatch,1
2023-02-12,Cleveland,Laptop,16


## Exercițiul 3 — Pivot table cu medie
- Creează un **tabel pivot** cu `values='Vânzări'`, `index='Oraș'`, `columns='Produs'`, `aggfunc='mean'`.
- **Cerințe:**
  1. Sortează rândurile descrescător după media totală pe oraș.
  2. Adaugă **margins** (totaluri) și formatează numeric (de ex. două zecimale) pentru afișare.
  3. Compară rezultatul cu o soluție `groupby` echivalentă (fără a le imprima simultan dacă e prea mare).

In [156]:
pivot = pd.pivot_table(
    df,
    values="Sales",
    index="City",
    columns="Product",
    aggfunc="mean",
    margins=True
)
pivot = pivot.sort_values(by="All", ascending=False)
pivot = pivot.round(2)

pivot

Product,Bluetooth Speaker,Desktop PC,Headphones,Keyboard,Laptop,Monitor,Mouse,Printer,Smartphone,Smartwatch,Tablet,Webcam,All
City,,,,,,,,,,,,,
Washington D.C.,NaN,15.00,NaN,NaN,NaN,18.00,NaN,NaN,17.00,NaN,NaN,24.00,18.83
New York,NaN,24.00,22.50,10.00,NaN,18.00,NaN,24.0,NaN,NaN,15.5,9.00,17.10
Baltimore,20.0,NaN,15.00,NaN,NaN,5.00,22.0,NaN,17.00,NaN,14.0,NaN,15.62
Boston,23.0,13.00,17.00,16.00,13.00,NaN,NaN,NaN,23.00,1.0,25.0,1.00,14.90
Providence,14.0,10.75,NaN,10.00,12.00,NaN,21.0,25.0,NaN,NaN,NaN,25.00,14.73
All,16.0,13.11,15.22,8.29,11.75,13.67,17.0,14.0,12.77,12.5,14.8,16.31,13.78
Cleveland,12.0,8.00,9.33,NaN,19.00,NaN,13.0,7.0,NaN,13.0,NaN,19.00,13.08
Hartford,NaN,NaN,NaN,NaN,6.33,18.00,25.0,NaN,10.00,NaN,4.0,14.00,11.27
Pittsburgh,NaN,NaN,NaN,7.00,NaN,8.00,NaN,3.0,10.00,NaN,NaN,17.33,11.25


In [157]:
grouped = df.groupby(["City", "Product"])["Sales"].mean().unstack()
grouped = grouped.round(2)

grouped

Product,Bluetooth Speaker,Desktop PC,Headphones,Keyboard,Laptop,Monitor,Mouse,Printer,Smartphone,Smartwatch,Tablet,Webcam
City,,,,,,,,,,,,
Baltimore,20.0,NaN,15.00,NaN,NaN,5.0,22.0,NaN,17.0,NaN,14.0,NaN
Boston,23.0,13.00,17.00,16.0,13.00,NaN,NaN,NaN,23.0,1.0,25.0,1.00
Buffalo,NaN,NaN,NaN,2.5,NaN,15.0,NaN,NaN,11.5,23.0,NaN,11.00
Cleveland,12.0,8.00,9.33,NaN,19.00,NaN,13.0,7.0,NaN,13.0,NaN,19.00
Hartford,NaN,NaN,NaN,NaN,6.33,18.0,25.0,NaN,10.0,NaN,4.0,14.00
New York,NaN,24.00,22.50,10.0,NaN,18.0,NaN,24.0,NaN,NaN,15.5,9.00
Philadelphia,11.0,NaN,NaN,NaN,NaN,NaN,4.0,12.5,9.5,NaN,NaN,NaN
Pittsburgh,NaN,NaN,NaN,7.0,NaN,8.0,NaN,3.0,10.0,NaN,NaN,17.33
Providence,14.0,10.75,NaN,10.0,12.00,NaN,21.0,25.0,NaN,NaN,NaN,25.00


## Exercițiul 4 — Gestionarea datelor lipsă
- Introdu intenționat **valori lipsă** (`NaN`) în coloana `Vânzări` pentru un subset de rânduri.
- **Cerințe:**
  1. Calculează `df.isnull().sum()` și identifică proporția de valori lipsă pe coloane.
  2. Aplică două strategii și compară rezultatele: `dropna()` vs. `fillna()` (de exemplu, cu **media/mediana pe grup** `Oraș-Produs`).
  3. Documentează într-un comentariu avantajele și dezavantajele fiecărei strategii în contextul setului tău de date.

In [158]:
if df.isnull().sum()["Sales"] < 5:
    df.loc[df.sample(5).index, "Sales"] = None 

print(df.isnull().sum())
print()
print(df.isnull().mean())

dropped = df.dropna(subset=["Sales"])
dropped

City       0
Product    0
Sales      5
dtype: int64

City       0.000000
Product    0.000000
Sales      0.055556
dtype: float64


,City,Product,Sales
2023-01-01,Buffalo,Webcam,11.0
2023-01-02,Pittsburgh,Monitor,8.0
2023-01-03,Baltimore,Mouse,22.0
2023-01-04,Baltimore,Monitor,5.0
2023-01-05,Providence,Desktop PC,25.0
...,...,...,...
2023-03-27,Hartford,Webcam,19.0
2023-03-28,Philadelphia,Smartphone,9.0
2023-03-29,Hartford,Webcam,9.0
2023-03-30,Cleveland,Headphones,7.0


In [159]:
if df.isnull().sum()["Sales"] < 5:
    df.loc[df.sample(5).index, "Sales"] = None 

filled = df.copy()
filled["Sales"] = filled.groupby(["City", "Product"])["Sales"].transform(
    lambda x: x.fillna(x.mean())
)

filled

,City,Product,Sales
2023-01-01,Buffalo,Webcam,11.0
2023-01-02,Pittsburgh,Monitor,8.0
2023-01-03,Baltimore,Mouse,22.0
2023-01-04,Baltimore,Monitor,5.0
2023-01-05,Providence,Desktop PC,25.0
...,...,...,...
2023-03-27,Hartford,Webcam,19.0
2023-03-28,Philadelphia,Smartphone,9.0
2023-03-29,Hartford,Webcam,9.0
2023-03-30,Cleveland,Headphones,7.0


In [160]:
filled = df.copy()
print(filled["Sales"].isnull().sum(), dropped.shape, filled.shape)

5 (85, 3) (90, 3)


## Exercițiul 5 — Funcție de curățare și pipeline de procesare
- Scrie o **funcție** `clean_data(df)` care:
  - elimină rândurile invalide (ex: `dropna` pe coloane critice),
  - **convertește tipurile** (ex: `Vânzări` → `float`),
  - **îmblânzește outlier-ii** (ex: `clip` pe percentile).
- Aplică funcția într-un **pipeline** Pandas (de ex.: `df.pipe(clean_data)`; poți compune cu alte funcții).
- **Cerințe:**
  1. Returnează un `DataFrame` curat, cu index temporal coerent.
  2. Arată fluxul înainte/după (dimensiuni, număr de valori lipsă, interval de date).
  3. Pregătește funcția pentru **reutilizare** (docstring, validări de intrare).


In [165]:
import pandas as pd
from pandas import DataFrame

def clean_data(df: DataFrame):
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")

    df = df.dropna(subset=["Sales"]).copy()
    df["Sales"] = df["Sales"].astype(float)

    # dangerous, multiple runs removes 5% from each side each run
    lower, upper = df["Sales"].quantile([0.05, 0.95])
    df["Sales"] = df["Sales"].clip(lower, upper)

    return df

new_df = df.pipe(clean_data)

new_df


,City,Product,Sales
2023-01-01,Buffalo,Webcam,11.0
2023-01-02,Pittsburgh,Monitor,8.0
2023-01-03,Baltimore,Mouse,22.0
2023-01-04,Baltimore,Monitor,5.0
2023-01-05,Providence,Desktop PC,25.0
...,...,...,...
2023-03-27,Hartford,Webcam,19.0
2023-03-28,Philadelphia,Smartphone,9.0
2023-03-29,Hartford,Webcam,9.0
2023-03-30,Cleveland,Headphones,7.0
